<a href="https://colab.research.google.com/github/sakuna47/Stock_Price_Prediction-/blob/Backend/Stock_Price_Prediction_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Enhanced Stock Price Prediction Model
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:

# =============================================================================
# CONFIGURATION
# =============================================================================
TICKER = "AAPL"  # Change this to any ticker you want
START_DATE = "2015-01-01"
END_DATE = "2025-06-25"
PREDICTION_DAYS_AHEAD = 7  # How many days ahead to predict (1=next day, 5=next week, etc.)
# =============================================================================


class EnhancedStockPredictionModel:
    def __init__(self, ticker=TICKER, start_date=START_DATE, end_date=END_DATE, prediction_horizon=PREDICTION_DAYS_AHEAD):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.prediction_horizon = prediction_horizon
        self.data = None
        self.model = None
        self.scaler = None
        self.feature_names = None
        self.target_scaler = None  # Add target scaling


    def download_data(self):
        """Download stock data with proper error handling"""
        print(f"Downloading {self.ticker} data from {self.start_date} to {self.end_date}")
        try:
            self.data = yf.download(self.ticker, start=self.start_date, end=self.end_date, progress=False)
            if self.data.empty:
                return False

            # Handle MultiIndex columns
            if isinstance(self.data.columns, pd.MultiIndex):
                self.data.columns = self.data.columns.get_level_values(0)

            # Remove duplicates
            self.data = self.data.loc[:, ~self.data.columns.duplicated()]

            print(f"Data shape: {self.data.shape}")
            print(f"Date range: {self.data.index[0]} to {self.data.index[-1]}")
            return True
        except Exception as e:
            print(f"Error downloading data: {e}")
            return False


     def create_enhanced_features(self):
        """Create enhanced features with better predictive power"""
        df = self.data.copy()

        # 1. RETURN-BASED FEATURES (more stationary)
        for period in [1, 2, 3, 5, 10, 20]:
            df[f'Returns_{period}d'] = df['Close'].pct_change(period)

        # 2. LOG FEATURES (help with non-stationarity)
        df['Log_Price'] = np.log(df['Close'])
        df['Log_Volume'] = np.log(df['Volume'] + 1)

        # 3. VOLATILITY FEATURES (multiple timeframes)
        for window in [5, 10, 20, 30]:
            returns = df['Close'].pct_change(1)
            df[f'Volatility_{window}'] = returns.rolling(window=window).std().shift(1)
            df[f'Volatility_Rank_{window}'] = (df[f'Volatility_{window}'].rolling(window=window*2)
                                              .rank(pct=True).shift(1))

        # 4. MOMENTUM INDICATORS
        for period in [5, 10, 20, 50]:
            df[f'Momentum_{period}'] = (df['Close'].shift(1) / df['Close'].shift(period+1)) - 1
            df[f'ROC_{period}'] = df['Close'].pct_change(period).shift(1)

        # 5. MOVING AVERAGE FEATURES (relative positioning)
        for window in [5, 10, 20, 50, 200]:
            ma = df['Close'].rolling(window=window).mean()
            df[f'Price_MA_Ratio_{window}'] = (df['Close'].shift(1) / ma.shift(1))
            df[f'MA_Slope_{window}'] = ((ma.shift(1) - ma.shift(6)) / ma.shift(6)).shift(1)

        # 6. BOLLINGER BANDS
        for window in [10, 20]:
            ma = df['Close'].rolling(window=window).mean()
            std = df['Close'].rolling(window=window).std()
            df[f'BB_Upper_{window}'] = ma + (2 * std)
            df[f'BB_Lower_{window}'] = ma - (2 * std)
            df[f'BB_Position_{window}'] = ((df['Close'].shift(1) - df[f'BB_Lower_{window}'].shift(1)) /
                                          (df[f'BB_Upper_{window}'].shift(1) - df[f'BB_Lower_{window}'].shift(1)))

